# Imports

In [131]:
import warnings
warnings.simplefilter('ignore')
from datetime import timedelta
from itertools import product
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
from rectools import Columns
from rectools.models import PopularModel, PopularInCategoryModel
from rectools.metrics import calc_metrics, MAP
from rectools.dataset import Dataset, Interactions
from rectools.model_selection import TimeRangeSplitter
from models.popular_among_n_unique import PopularAmongNUniqueUsers

# Loading data

In [2]:
interactions = pd.read_csv("data/interactions.csv")
items = pd.read_csv("data/items.csv")
users = pd.read_csv("data/users.csv") 

In [3]:
sample_submission = pd.read_csv("data/sample_submission.csv")

In [4]:
sample_submission

,user_id,item_id
0,3,"[9728, 15297, 10440, 14488, 13865, 12192, 341,..."
1,11,"[9728, 15297, 10440, 14488, 13865, 12192, 341,..."
2,29,"[9728, 15297, 10440, 14488, 13865, 12192, 341,..."
3,30,"[9728, 15297, 10440, 14488, 13865, 12192, 341,..."
4,33,"[9728, 15297, 10440, 14488, 13865, 12192, 341,..."
...,...,...
193108,1097527,"[9728, 15297, 10440, 14488, 13865, 12192, 341,..."
193109,1097537,"[9728, 15297, 10440, 14488, 13865, 12192, 341,..."
193110,1097538,"[9728, 15297, 10440, 14488, 13865, 12192, 341,..."
193111,1097544,"[9728, 15297, 10440, 14488, 13865, 12192, 341,..."


# Data preprocessing

In [5]:
interactions.rename(
    columns={
        'track_id' : Columns.Item,
        'last_watch_dt' : Columns.Datetime,
        'total_dur' : Columns.Weight
    },
    inplace=True
)

In [6]:
_, bins = pd.qcut(items["release_year"], 10, retbins=True)
labels = bins[:-1]

year_feature = pd.DataFrame(
    {
        Columns.Item: items[Columns.Item],
        "value": pd.cut(items["release_year"], bins=bins, labels=bins[:-1]),
        "feature": "release_year",
    }
)
year_feature.head()

,item_id,value,feature
0,10711,1983.0,release_year
1,2508,2012.0,release_year
2,10716,2009.0,release_year
3,7868,2014.0,release_year
4,16268,1897.0,release_year


In [7]:
items['genre'] = items['genres'].str.split(",")

In [8]:
genre_feature = items[[Columns.Item, "genre"]].explode("genre")
genre_feature.columns = [Columns.Item, "value"]
genre_feature["feature"] = "genre"
genre_feature.head()

,item_id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre


In [9]:
item_feat = pd.concat([genre_feature, year_feature])
item_feat = item_feat[item_feat[Columns.Item].isin(interactions[Columns.Item])]

In [127]:
items['director'] = items['directors'].str.split(",")
director_feature = items[[Columns.Item, "director"]].explode("director")
director_feature.columns = [Columns.Item, "value"]
director_feature["feature"] = "director"
director_feature.tail()

,item_id,value,feature
15960,10632,Амир Камдин,director
15960,10632,Эрик Эгер,director
15961,4538,Марк О’Коннор,director
15961,4538,Конор МакМахон,director
15962,3206,Михаил Миронов,director


In [128]:
item_feat = pd.concat([genre_feature, year_feature, director_feature])
item_feat = item_feat[item_feat[Columns.Item].isin(interactions[Columns.Item])]

In [129]:
item_feat

,item_id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre
...,...,...,...
15960,10632,Амир Камдин,director
15960,10632,Эрик Эгер,director
15961,4538,Марк О’Коннор,director
15961,4538,Конор МакМахон,director


In [11]:
n_folds = 3
unit = "D"
n_units = 7
# last_date = interactions[Columns.Datetime].max().normalize()
last_date = pd.to_datetime('15-08-2021', format='%d-%m-%Y')


start_date = last_date - pd.Timedelta(n_folds * n_units - 1, unit=unit)
periods = n_folds + 1
freq = f"{n_units}{unit}"
print(f"""
    start_date: {start_date}
    last_date: {last_date}
    periods: {periods}
    freq: {freq}
""")

date_range = pd.date_range(start=start_date, periods=periods, freq=freq, tz=last_date.tz)


    start_date: 2021-07-26 00:00:00
    last_date: 2021-08-15 00:00:00
    periods: 4
    freq: 7D



In [12]:
splitter = TimeRangeSplitter(date_range=date_range)
interactions_ = Interactions(interactions)

# Hypeparameter tuning and fitting simple Popular model

In [121]:
model_parameters = {
    "popularity": ["n_users", "n_interactions", "sum_weight", "mean_weight"], 
    "period" : [None] + [i for i in range(1, 22)]
}

In [122]:
parameter_grid = tuple(product(*list(model_parameters.values())))

In [123]:
folds = splitter.split(interactions_)

metrics = {
    'MAP@10' : MAP(k=10), 
}

results = {
    "model_name" : [],
    "map" : []
}

models = {}

In [124]:
for i, (train_ids, test_ids, _) in tqdm(enumerate(folds), total=splitter.get_n_splits(interactions_)):
    df_train = interactions_.df.iloc[train_ids]
    item_feat_train = item_feat[item_feat[Columns.Item].isin(df_train[Columns.Item])]

    dataset = Dataset.construct(
        df_train,
        None,
        item_feat_train,
        cat_item_features=['genre', 'release_year']
    )

    df_test = interactions_.df.iloc[test_ids][Columns.UserItem]
    test_users = np.unique(df_test[Columns.User])
    
    for params in tqdm(parameter_grid):
        model = PopularModel(popularity=params[0], period=pd.to_timedelta(params[1], unit='D'))
        model.fit(dataset)

        recos = model.recommend(
            users = test_users,
            dataset=dataset,
            k=10,
            filter_viewed=True
        )t

        metric_values = calc_metrics(
            metrics,
            reco=recos,
            interactions=df_test,
            prev_interactions=df_train
        )
        
        results['model_name'].append(f"{params[0]}_{params[1]}")
        results['map'].append(metric_values["MAP@10"])
        
        models[f"{params[0]}_{params[1]}"] = model

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/88 [00:00<?, ?it/s]

  0%|          | 0/88 [00:00<?, ?it/s]

  0%|          | 0/88 [00:00<?, ?it/s]

In [128]:
results_df = pd.DataFrame(data=results)

In [129]:
results_df.groupby("model_name").mean().sort_values('map', ascending=False)

,map
model_name,
n_users_5,0.096757
n_interactions_5,0.096757
n_interactions_4,0.096695
n_users_4,0.096695
n_interactions_3,0.096656
...,...
mean_weight_5,0.000106
mean_weight_6,0.000102
mean_weight_21,0.000098


In [136]:
dataset = Dataset.construct(
    interactions_df=interactions,
    user_features_df=None,
    item_features_df=item_feat,
    cat_item_features=['genre', 'release_year']
)

Модель с параметрами n_users, 5 дает наилучший прирост в метрике

In [161]:
submission_users = sample_submission[sample_submission['user_id'].isin(dataset.user_id_map.external_ids)]['user_id']

In [155]:
recs = models['n_users_5'].recommend(submission_users, dataset, 10, filter_viewed=True)

In [162]:
preds = []
default = [9728, 15297, 10440, 14488, 13865, 12192, 341, 4151, 3734, 512]

In [178]:
for i, user_id in tqdm(enumerate(sample_submission['user_id']), total=len(sample_submission)):
    items = list(recs[recs.user_id == user_id]['item_id'])
    
    if items: 
        preds.append(items)
    else:
        preds.append(default)

  0%|          | 0/193113 [00:00<?, ?it/s]

In [180]:
sample_submission['item_id'] = preds

In [184]:
sample_submission.to_csv("submissions/popular_model.csv", index=False)

# Hypeparameter tuning and fitting Popular among N percent of users model

In [89]:
popularity_types = ["n_users", "n_interactions", "sum_score", "mean_score"]

In [100]:
folds = splitter.split(interactions_)

metrics = {
    'MAP@10' : MAP(k=10), 
}

results = {
    "model_name" : [],
    "map" : []
}

models = {}

for i, (train_ids, test_ids, _) in tqdm(enumerate(folds), total=splitter.get_n_splits(interactions_)):
    df_train = interactions_.df.iloc[train_ids]
    item_feat_train = item_feat[item_feat[Columns.Item].isin(df_train[Columns.Item])]

    dataset = Dataset.construct(
        df_train,
        None,
        item_feat_train,
        cat_item_features=['genre', 'release_year']
    )

    df_test = interactions_.df.iloc[test_ids][Columns.UserItem]
    test_users = np.unique(df_test[Columns.User])
    
    for popularity_type in popularity_types:
        model = PopularAmongNUniqueUsers(popularity_type=popularity_type,start_from=None).fit(dataset, 650_000)

        recos = model.recommend(
            users_ids = test_users,
            k=10,
        )

        metric_values = calc_metrics(
            metrics,
            reco=recos,
            interactions=df_test,
            prev_interactions=df_train
        )
        
        results['model_name'].append(f"{popularity_type}")
        results['map'].append(metric_values["MAP@10"])
        
        models[f"{popularity_type}"] = model

  0%|          | 0/3 [00:00<?, ?it/s]

Getting recommendations :   0%|          | 0/94910 [00:00<?, ?it/s]

Getting recommendations :   0%|          | 0/94910 [00:00<?, ?it/s]

Getting recommendations :   0%|          | 0/94910 [00:00<?, ?it/s]

Getting recommendations :   0%|          | 0/94910 [00:00<?, ?it/s]

...

Getting recommendations :   0%|          | 0/98184 [00:00<?, ?it/s]

Getting recommendations :   0%|          | 0/103511 [00:00<?, ?it/s]

Getting recommendations :   0%|          | 0/103511 [00:00<?, ?it/s]

Getting recommendations :   0%|          | 0/103511 [00:00<?, ?it/s]

Getting recommendations :   0%|          | 0/103511 [00:00<?, ?it/s]

In [103]:
results_df = pd.DataFrame(data=results)
results_df

,model_name,map
0,n_users,0.0
1,n_interactions,0.0
2,sum_score,0.0
3,mean_score,0.0
4,n_users,0.0
5,n_interactions,0.0
6,sum_score,0.0
7,mean_score,0.0
8,n_users,0.0
9,n_interactions,0.0


P.S не знаю почему модель не дает прирост в метрике :(

# Hyperparameter tuning and fitting Most Polular in category models on item features

In [152]:
model_parameters = {
    "period" : [None] + [i for i in range(1, 10)],
    "category_feature" : ['genre', 'release_year', 'director'],
    "n_categories" : [3,4,5]
}

In [153]:
parameter_grid = list(product(*list(model_parameters.values())))

In [154]:
folds = splitter.split(interactions_)

metrics = {
    'MAP@10' : MAP(k=10), 
}

results = {
    "model_name" : [],
    "map" : []
}

models = {}

for i, (train_ids, test_ids, _) in tqdm(enumerate(folds), total=splitter.get_n_splits(interactions_)):
    df_train = interactions_.df.iloc[train_ids]
    item_feat_train = item_feat[item_feat[Columns.Item].isin(df_train[Columns.Item])]

    dataset = Dataset.construct(
        df_train,
        None,
        item_features_df=item_feat_train,
        cat_item_features=['genre', 'release_year', 'director']
    )

    df_test = interactions_.df.iloc[test_ids][Columns.UserItem]
    test_users = np.unique(df_test[Columns.User])
    
    for params in tqdm(parameter_grid):
        model = PopularInCategoryModel(period=pd.to_timedelta(params[0], unit='D'),
                                       category_feature=params[1],
                                       n_categories=params[2])
        model.fit(dataset)

        recos = model.recommend(
            users = test_users,
            dataset=dataset,
            k=10,
            filter_viewed=True
        )

        metric_values = calc_metrics(
            metrics,
            reco=recos,
            interactions=df_test,
            prev_interactions=df_train
        )
        
        results['model_name'].append(f"{params[0]}_{params[1]}_{params[2]}")
        results['map'].append(metric_values["MAP@10"])
        
        models[f"{params[0]}_{params[1]}_{params[2]}"] = model

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/90 [00:00<?, ?it/s]

  0%|          | 0/90 [00:00<?, ?it/s]

  0%|          | 0/90 [00:00<?, ?it/s]

In [156]:
results_df = pd.DataFrame(data=results)

In [159]:
results_df.groupby("model_name").mean().sort_values('map', ascending=False)

,map
model_name,
7_release_year_4,0.075586
3_genre_5,0.075541
4_genre_5,0.075461
5_genre_5,0.075409
8_release_year_4,0.075399
...,...
3_director_3,0.064431
2_director_3,0.064430
5_director_3,0.063816


In [165]:
dataset = Dataset.construct(
        interactions,
        None,
        item_features_df=item_feat,
        cat_item_features=['genre', 'release_year', 'director']
    )

In [172]:
recs = models['7_release_year_4'].recommend(submission_users, dataset, 10, filter_viewed=True)

In [ ]:
preds = []

for i, user_id in tqdm(enumerate(sample_submission['user_id']), total=len(sample_submission)):
    items = list(recs[recs.user_id == user_id]['item_id'])
    
    if items: 
        preds.append(items)
    else:
        preds.append(default)

  0%|          | 0/193113 [00:00<?, ?it/s]

In [ ]:
sample_submission['item_id'] = preds

In [168]:
sample_submission.to_csv("submissions/popular_in_category_items.csv", index=False)

,user_id,item_id,score,rank
0,699317,15297,18004.0,1
1,699317,598,3802.0,2
2,699317,8636,1546.0,3
3,699317,13865,7168.0,4
4,699317,10152,15099.0,5
...,...,...,...,...
987305,541430,5434,3538.0,6
987306,541430,15815,1200.0,7
987307,541430,3190,14326.0,8
987308,541430,3043,6905.0,9


In [169]:
sample_submission

,user_id,item_id
0,3,"[9728, 15297, 10440, 14488, 13865, 12192, 341,..."
1,11,"[9728, 15297, 10440, 14488, 13865, 12192, 341,..."
2,29,"[9728, 15297, 10440, 14488, 13865, 12192, 341,..."
3,30,"[9728, 15297, 10440, 14488, 13865, 12192, 341,..."
4,33,"[9728, 15297, 10440, 14488, 13865, 12192, 341,..."
...,...,...
193108,1097527,"[9728, 15297, 10440, 14488, 13865, 12192, 341,..."
193109,1097537,"[9728, 15297, 10440, 14488, 13865, 12192, 341,..."
193110,1097538,"[9728, 15297, 10440, 14488, 13865, 12192, 341,..."
193111,1097544,"[9728, 15297, 10440, 14488, 13865, 12192, 341,..."
